In [1]:
#!/usr/bin/Rscript
#############################################################
options(stringsAsFactors = FALSE)
##############################################################

suppressMessages(
    {

    library(tidyverse)
    library(data.table)
    library(dynamicTreeCut)
    library(fastcluster)
    library(WGCNA)
    library(flashClust)
    library(lattice)
    library(latticeExtra)
    library(gridExtra)
    library(grid)
    library(ggplot2)

    #enableWGCNAThreads(15)
    allowWGCNAThreads(15)

    }
                )


save_plot <- function(p, fn, w, h){
    for(ext in c(".pdf", ".png")){
        ggsave(filename=paste0(fn,ext), plot=p, width=w, height=h)
    }
}


dir.create('../_m/')
setwd('../_m/')

Allowing multi-threading with up to 15 threads.


Warning message in dir.create("../_m/"):
“'../_m' already exists”


In [2]:

counts_path <- Sys.glob('../../../../jhpce_data/analysis/*/residualized_expression.tsv')
#counts_path <- counts_path[1]


region <- gsub('.*analysis/|/residu.*','',counts_path)
region

[1] "caudate"     "dlpfc"       "hippocampus"

In [3]:
rdata_path <- Sys.glob('../../../../jhpce_data/analysis/*/voomSVA.RData')
#rdata_path <- rdata_path[1]
rdata_path
#load(rdata_path)

[1] "../../../../jhpce_data/analysis/caudate/voomSVA.RData"    
[2] "../../../../jhpce_data/analysis/dlpfc/voomSVA.RData"      
[3] "../../../../jhpce_data/analysis/hippocampus/voomSVA.RData"

In [4]:
#process the dataframes
process_data <- function(data, group) {
  data %>%
        dplyr::select(row.names(group)) %>%
        t() %>%
        .[goodSamplesGenes(., verbose = 3)$goodSamples, goodSamplesGenes(., verbose = 3)$goodGenes] %>%
        t()
    }

# split the expr data between groups (ie. case vs control, male vs female etc)
separate_data <- function(data,group) {
    df_tmp <- table_all %>% 
                        t() %>%
                        as.data.frame() %>%
                        tibble::rownames_to_column(var = 'row_name') %>%
                        filter(row_name %in% row.names(group)) %>%
                        tibble::column_to_rownames(var = 'row_name')
    return(df_tmp)
    
    }

# Define a function to get the max value between two vectors
max_vector <- function(x, y) { 
    if (max(x) > max(y)) x else y
        }



### WGNCA functions:

create_multiExpr <- function(datExpr) { 
    multi <- list(data = datExpr)
    names(multi$data) <- names(datExpr)
    rownames(multi$data) <- dimnames(datExpr)[[1]]
    return(multi)
        }


create_multiME <- function(Eigengene, datExpr) {
    multiME <- list(data = Eigengene)
    rownames(multiME$data) <- dimnames(datExpr)[[1]]
    return(multiME)
        }


create_pTable_CountTbl <- function(SetModColors, ConsModColors) {
        No.SetMods <- nlevels(SetModColors)
        No.ConsMods <- nlevels(ConsModColors)
        pTable <- matrix(0, nrow = No.SetMods, ncol = No.ConsMods)
        CountTbl <- matrix(0, nrow = No.SetMods, ncol = No.ConsMods)

        for (smod in 1:No.SetMods) {
            for (cmod in 1:No.ConsMods) {
                SetMembers <- (SetModColors == levels(SetModColors)[smod])
                ConsMembers <- (ConsModColors == levels(ConsModColors)[cmod])
                pTable[smod, cmod] <- -log(fisher.test(SetMembers, ConsMembers, alternative = "greater")$p.value)
                CountTbl[smod, cmod] <- sum(SetMembers & ConsMembers)
            }
          }

        pTable[is.infinite(pTable)] <- 1.3 * max(pTable[is.finite(pTable)])
        pTable[pTable > 50] <- 50
        list(pTable = pTable, CountTbl = CountTbl)
        }

calculate_PercentageTbl <- function(CountTbl, SetModColors) {
        No.SetMods <- nrow(CountTbl)
        PercentageTbl <- CountTbl
        for (smod in 1:No.SetMods) {
            PercentageTbl[smod, ] <- as.integer(PercentageTbl[smod, ] / sum(PercentageTbl[smod, ]) * 100)
          }
        PercentageTbl
        }

write_module_eigengenes <- function(datExpr, dynamicColors, set_name, file_prefix) {
        PCA <- moduleEigengenes(datExpr, colors = dynamicColors)
        ME <- PCA$eigengenes
        row.names(ME) <- row.names(datExpr)
        write.table(ME, paste0(save_path,sprintf("%s-Module_eigengenes_%s.tsv", file_prefix, set_name)), sep="\t", row.names=TRUE)
        ME
    }

write_kME_table <- function(datExpr, ME, dynamicColors, set_name, file_prefix) {
        colors <- names(table(dynamicColors))
        geneModuleMembership <- signedKME(datExpr, ME)
        colnames(geneModuleMembership) <- paste0("PC", colors, ".cor")
        MMPvalue <- corPvalueStudent(as.matrix(geneModuleMembership), nrow(datExpr))
        colnames(MMPvalue) <- paste0("PC", colors, ".pval")
        Gene <- colnames(datExpr)
        kMEtable <- cbind(Gene, Gene, dynamicColors)
        for (i in 1:length(colors)) {
            kMEtable <- cbind(kMEtable, geneModuleMembership[, i], MMPvalue[, i])
          }
        colnames(kMEtable) <- c("PSID", "Gene", "Module", sort(c(colnames(geneModuleMembership), colnames(MMPvalue))))
        write.table(kMEtable, paste0(save_path,sprintf("%s-Module_kME_%s.tsv", file_prefix, set_name)), row.names = FALSE, sep = "\t")
        kMEtable
}

In [5]:
#######################
# select here important WGCNA paramaters
setLabels <- c('Male','Female')
setA <- setLabels[1]
setB <- setLabels[2]

power=15 #change accordingly to each experiment!

#Choose the minimum module size (30~50)

minModuleSize = 50

#signed network
#corType = "bicor"
#mergeCutHeight = 0.15

In [6]:
for (i in seq_along(counts_path)){
    
    load(rdata_path[i])
    
    dir.create(region[i])
    
    metadata <- v$targets %>% 
                        as.data.frame()
    
    # CTL <- metadata %>% 
    #                 filter(Dx == 'Control')
    # SZD <- metadata %>% 
    #                 filter(Dx != 'Control')
    All <- metadata
    
    for (sample_filter in c('All')){

        print('############################## start')

        
        save_path <- paste0(region[i],'/',sample_filter,'/')
        
        dir.create(save_path)
        
        filtered_metadata <- get(sample_filter)
        
        group_a <- filtered_metadata %>% 
                                    filter(Sex == 'M') %>% #male only
                                    filter(RNum != 'R3555') #remove this specific sample, huge outlier
        group_b <- filtered_metadata %>% 
                                    filter(Sex != 'M') %>% #female only 
                                    filter(!RNum %in% c('R12351', 'R12371'))  #remove these specific samples, huge outlier
    
        vsd<-fread(counts_path[i],header = T, na.strings = "",check.names = F) %>%
                                                tibble::column_to_rownames(var = "feature_id") #%>%
                                                #slice_head(n=2000)
    
        
        # Process data for group_a and group_b
        vsd_group_a <- process_data(vsd, group_a)
        vsd_group_b <- process_data(vsd, group_b)
        
        print(paste0('group_a after goodSamplesGenes(): ', dim(vsd_group_a)))
        print(paste0('group_b after goodSamplesGenes(): ', dim(vsd_group_b)))

        # Merge the processed data
        table_all <- merge(vsd_group_a, vsd_group_b, by = "row.names") %>%
                                                tibble::column_to_rownames('Row.names') 


        datExprA <- separate_data(tabble_all, group_a)
        datExprB <- separate_data(tabble_all, group_b)
        datExprAll <- table_all %>%
                                t() %>%
                                as.data.frame()
        
        print(paste0('datExprA after goodSamplesGenes(): ', dim(datExprA)))
        print(paste0('datExprB after goodSamplesGenes(): ', dim(datExprB)))
        print(paste0('datExprAll : ', dim(datExprAll)))

        sampleTreeA <- flashClust(dist(datExprA), method="average")
        sampleTreeB <- flashClust(dist(datExprB), method="average")
        sampleTreeAll <- flashClust(dist(datExprAll), method="average")


        ###save as .PDF file 
        pdf(paste0(save_path,'1a-Dist_clust_',sample_filter,'.pdf'),height=10,width = 15)
        par(mar=c(2,4,1,0), mfrow=c(1,2), oma=c(2,0,4,0), cex=0.5)
        plot(sampleTreeA, main=paste0(setA, ' ', sample_filter), sub="", cex.lab=1.2, cex.axis=1.2, cex.main=1.7, ylim=20)
        plot(sampleTreeB, main=paste0(setB, ' ',sample_filter), sub="", cex.lab=1.2, cex.axis=1.2, cex.main=1.7, ylim=20)
        par(cex=1)
        title(main="Sample Clustering Based on Distance ",outer=TRUE)
        dev.off()


        ###save as .PDF file 
        pdf(paste0(save_path,'1a-Dist_clust_all_',sample_filter,'.pdf'),height=10,width = 15)
        par(cex=0.5)
        plot(sampleTreeAll, main='', sub="", cex.lab=1.2, cex.axis=1.2, cex.main=1.7, ylim=20)
        title(main="Sample Clustering Based on Distance ",outer=TRUE)
        dev.off()
        
        
        ##calculate the scale-free topology index
        powers1 = c(seq(1,30,by = 1))
        sink('pickSoftThreshold.log')
        RpowerTableA <- pickSoftThreshold(datExprA, powerVector = powers1, RsquaredCut = 0.85, verbose = 1)
        RpowerTableB <- pickSoftThreshold(datExprB, powerVector = powers1, RsquaredCut = 0.85, verbose = 1)
        sink()
        
        
        #save results into a dataframe
        RpowerTableA$fitIndices %>%
                fwrite(paste0(save_path,'0-',setA,'_scale_free_metrics.tsv'),quote=F,sep='\t',row.names=F)
        RpowerTableB$fitIndices %>%
                fwrite(paste0(save_path,'0-',setB,'_scale_free_metrics.tsv'),quote=F,sep='\t',row.names=F)
        
        pdf(paste0(save_path,'2-scalefree_connectivity-analysis.pdf'))
                    
        par(mfcol = c(2,2), mar = c(4.2, 4.5 , 2.2, 0.5), oma=c(0,0,2,0))
        cex1 = 0.7
                
        # Scale independence
        plot_sign <- -sign(RpowerTableB$fitIndices[,3]) * RpowerTableB$fitIndices[,2]
        plot_sign <- max_vector(plot_sign, -sign(RpowerTableA$fitIndices[,3]) * RpowerTableA$fitIndices[,2])
        plot(plot_sign, xlab="Soft Threshold (Power)", ylab=bquote("Signed" ~(R^2)), 
             type="n", main="Scale Free Topology Model Fit")
        text(RpowerTableA$fitIndices[,1], -sign(RpowerTableA$fitIndices[,3]) * RpowerTableA$fitIndices[,2],
             labels=powers1, cex=0.85, col="blue")
        text(RpowerTableB$fitIndices[,1], -sign(RpowerTableB$fitIndices[,3]) * RpowerTableB$fitIndices[,2],
             labels=powers1, cex=0.85, col="red")
        abline(h=0.8)
        legend("bottomright", legend = setLabels, col = c("blue","red"), pch = 20, cex = 0.65)

        # Mean connectivity
        plot_mean <- max_vector(RpowerTableB$fitIndices[,5], RpowerTableA$fitIndices[,5])
        plot(RpowerTableA$fitIndices[,1], plot_mean, xlab="Soft Threshold (Power)", ylab="Mean Connectivity",
             type="n", main="Mean Connectivity")
        text(RpowerTableA$fitIndices[,1], RpowerTableA$fitIndices[,5], labels=powers1, cex=0.85, col="blue")
        text(RpowerTableB$fitIndices[,1], RpowerTableB$fitIndices[,5], labels=powers1, cex=0.85, col="red")
        legend("topright", legend = setLabels, col = c("blue","red"), pch = 20, cex = 0.65)

        # Median connectivity
        plot_median <- max_vector(RpowerTableB$fitIndices[,6], RpowerTableA$fitIndices[,6])
        plot(RpowerTableA$fitIndices[,1], plot_median, xlab="Soft Threshold (Power)", ylab="Median Connectivity",
             type="n", main="Median Connectivity")
        text(RpowerTableA$fitIndices[,1], RpowerTableA$fitIndices[,6], labels=powers1, cex=0.85, col="blue")
        text(RpowerTableB$fitIndices[,1], RpowerTableB$fitIndices[,6], labels=powers1, cex=0.85, col="red")
        legend("topright", legend = setLabels, col = c("blue","red"), pch = 20, cex = 0.65)

        # Max connectivity
        plot_max <- max_vector(RpowerTableB$fitIndices[,7], RpowerTableA$fitIndices[,7])
        plot(RpowerTableA$fitIndices[,1], plot_max, xlab="Soft Threshold (Power)", ylab="Max Connectivity",
             type="n", main="Max Connectivity")
        text(RpowerTableA$fitIndices[,1], RpowerTableA$fitIndices[,7], labels=powers1, cex=0.85, col="blue")
        text(RpowerTableB$fitIndices[,1], RpowerTableB$fitIndices[,7], labels=powers1, cex=0.85, col="red")
        legend("topright", legend = setLabels, col = c("blue","red"), pch = 20, cex = 0.65)

        dev.off()       
        
        
        # Topological Overlap Matrix (TOM) Modules Identification and Construction
        # TOM Module construction        
        Ad_A <- adjacency(datExprA, power = power)
        Ad_B <- adjacency(datExprB, power=power)
        TOMA_diss <- TOMdist(Ad_A)
        TOMB_diss<- TOMdist(Ad_B)
        geneTreeA <- flashClust(as.dist(TOMA_diss), method="average")
        geneTreeB <- flashClust(as.dist(TOMB_diss), method="average")

        dynamicModsA = cutreeDynamic(dendro=geneTreeA,  distM=TOMA_diss, deepSplit=2, pamRespectsDendro=FALSE, minClusterSize=minModuleSize)
        table(dynamicModsA)
        dynamicColorsA <- labels2colors(dynamicModsA)
        table(dynamicColorsA)
        dynamicModsB = cutreeDynamic(dendro=geneTreeB, distM=TOMB_diss, deepSplit=2, pamRespectsDendro=FALSE, minClusterSize=minModuleSize)
        table(dynamicModsB)
        dynamicColorsB <- labels2colors(dynamicModsB)
        table(dynamicColorsB)
        
        # Plot the TOM Clustering for each network
        sizeGrWindow(10,7)
        pdf(paste0(save_path,'3-clustering_tom-based.pdf'),width = 9,height = 9)
        cex=0.8
        plotDendroAndColors(geneTreeA, dynamicColorsA, dendroLabels=F, "Dynamic Tree Cut", hang = 0.03, addGuide = TRUE, guideHang = 0.05, main = setA)
        plotDendroAndColors(geneTreeB, dynamicColorsB, dendroLabels=F, "Dynamic Tree Cut", hang = 0.03, addGuide = TRUE, guideHang = 0.05, main = setB)
        dev.off()
        
        
        # Sets union and co-expression analysis (deepsplit = 2 ; mergecutheight = 0.15)
        
        nSets <- 2

        multiExpr <- lapply(list(datExprA, datExprB), create_multiExpr)
        expsize <- checkSets(multiExpr)
        expsize


        netConsensus <- blockwiseConsensusModules(multiExpr, corType = "bicor", networkType = "signed",
                                                  checkMissingData = T, minModuleSize = minModuleSize, 
                                                  power = power, deepSplit = 2, pamRespectsDendro = T, 
                                                  mergeCutHeight = 0.15, minKMEtoStay = 0.2, saveTOMs = T)


        Eigengene_A <- netConsensus$multiMEs[[1]]$data
        Eigengene_B <- netConsensus$multiMEs[[2]]$data
        
        multiME <- mapply(create_multiME, list(Eigengene_A, Eigengene_B), list(datExprA, datExprB), SIMPLIFY = FALSE)

        MEsize <- checkSets(multiME)
        
        #plot the eigengenes
        pdf(paste0(save_path,'4-eigengenesnetworks.pdf'),width = 12,height = 9)
        
        #colorBWR <- blueWhiteRed(50, gamma=3)
        cex=0.2
        plotEigengeneNetworks(multiME,setLabels=setLabels, letterSubPlots=T, plotDendrograms=T, 
                              plotHeatmaps=F, marDendro=c(1,3,1.2,2), marHeatmap=c(2,3,2.5,5), excludeGrey=T, 
                              colorLabels=T, signed=T, setMargins=F, plotAdjacency=T, printAdjacency=T, 
                              printPreservation=T)
        dev.off()
        
        

#         ##################################################

        consensus_colors <- netConsensus$colors
        SetModColors <- as.factor(dynamicColorsA)
        ConsModColors <- as.factor(consensus_colors)

        results <- create_pTable_CountTbl(SetModColors, ConsModColors)
        pTable <- results$pTable
        CountTbl <- results$CountTbl
        PercentageTbl <- calculate_PercentageTbl(CountTbl, SetModColors)

        SetModsTotals <- as.vector(table(SetModColors))
        ConsModTotals <- as.vector(table(ConsModColors))

        sizeGrWindow(10, 5)

        par(mfrow = c(1, 1), cex = 1.4 / 1.7, mar = c(8, 12, 2, 1) + 0.3)

        rownames(pTable) <- paste(setA,levels(SetModColors), ":", SetModsTotals, sep = "")
        colnames(pTable) <- paste("Consensus ", levels(ConsModColors), ": ", ConsModTotals, sep = "")
        colors <- colorRampPalette(c("yellow", "red"), space = "Lab") 
        grid <- expand.grid(1:nrow(CountTbl), 1:ncol(CountTbl))
        grid$Z <- as.numeric(CountTbl)
        names(grid) <- c("X", "Y", "Z")
        plotaConsensus <- levelplot(pTable, scales = list(x = list(rot = 90)), xlab = "", ylab = "", col.regions = colors, main = paste0(setA," modules vs Consensus Modules")) + layer(panel.text(X, Y, Z), data = grid)               

        

#         ################################################### 
      
        SetModColors <- as.factor(dynamicColorsB)
        ConsModColors <- as.factor(consensus_colors)

        results <- create_pTable_CountTbl(SetModColors, ConsModColors)
        pTable <- results$pTable
        CountTbl <- results$CountTbl
        PercentageTbl <- calculate_PercentageTbl(CountTbl, SetModColors)

        SetModsTotals <- as.vector(table(SetModColors))
        ConsModTotals <- as.vector(table(ConsModColors))
        
        
        sizeGrWindow(10, 5)
        par(mfrow = c(1, 1), cex = 1.4 / 1.7, mar = c(8, 12, 2, 1) + 0.3)

        rownames(pTable) <- paste(setB, levels(SetModColors), ": ", SetModsTotals, sep = "")
        colnames(pTable) <- paste("Consensus ", levels(ConsModColors), ": ", ConsModTotals, sep = "")
        colors <- colorRampPalette(c("yellow", "red"), space = "Lab") 
        grid <- expand.grid(1:nrow(CountTbl), 1:ncol(CountTbl))
        grid$Z <- as.numeric(CountTbl)
        names(grid) <- c("X", "Y", "Z")
        plotbConsensus <- levelplot(pTable, scales = list(x = list(rot = 90)), xlab = "", ylab = "", col.regions = colors, main = paste0(setB," modules vs Consensus Modules")) + layer(panel.text(X, Y, Z), data = grid)               

        
#         #########################################   
    
        sizeGrWindow(10, 5)
        par(mfrow = c(1, 1), cex = 1.4 / 1.7, mar = c(8, 12, 2, 1) + 0.3)

        rownames(pTable) <- paste(setB, levels(SetModColors), ": ", SetModsTotals, sep = "")
        colnames(pTable) <- paste(setA, levels(ConsModColors), ": ", ConsModTotals, sep = "")
        colors <- colorRampPalette(c("yellow", "red"), space = "Lab") 
        grid <- expand.grid(1:nrow(CountTbl), 1:ncol(CountTbl))
        grid$Z <- as.numeric(CountTbl)
        names(grid) <- c("X", "Y", "Z")
        plotsc <- levelplot(pTable, scales = list(x = list(rot = 90)), xlab = "", ylab = "", col.regions = colors, main = paste(setB, "Modules", "vs", setA, "Modules", sep = ' ')) + layer(panel.text(X, Y, Z), data = grid)

        require(grid)
        grid.newpage()
        pdf(paste0(save_path,'5-consensusmodules_plot.pdf'), width = 15, height = 12)
        masterLayout <- grid.layout(nrow = 2, ncol = 2)
        vp1 <- viewport(layout.pos.row = 1:2, layout.pos.col = 1, name = "vp1")  
        vp2 <- viewport(layout.pos.row = 1, layout.pos.col = 2, name = "vp2")     
        vp3 <- viewport(layout.pos.row = 2, layout.pos.col = 2, name = "vp3")     
        pushViewport(vpTree(viewport(layout = masterLayout, name = "master"), vpList(vp1, vp2, vp3)))
        seekViewport("master")
        print(plotsc, draw.in = "vp1")
        print(plotaConsensus, draw.in = "vp2")
        print(plotbConsensus, draw.in = "vp3")

        dev.off()
        
        
        ####################################################
        
        # Calculate the Module Preservation Statistics (medianRank and zSummary)

        # Set names for multiExpr and A_colors
        names(multiExpr) <- setLabels
        A_colors <- setNames(lapply(setA, function(x) dynamicColorsA), setA)

        # Calculate module preservation
        system.time({
          mp2 <- modulePreservation(multiExpr,
                                    A_colors,
                                    dataIsExpr = TRUE,
                                    referenceNetworks = 1,
                                    nPermutations = 100,
                                    randomSeed = 1,
                                    quickCor = 0,
                                    verbose = 3,
                                    networkType = "signed"
          )
        })
        # Compare preservation to quality
        ref=1
        test=2
        stasObs = cbind(mp2$quality$observed[[ref]][[test]][,-1], mp2$preservation$observed[[ref]][[test]][,-1])
        stastZ = cbind(mp2$quality$Z[[ref]][[test]][,-1], mp2$preservation$Z[[ref]][[test]][,-1])

        print(cbind(stasObs[,c("medianRank.pres", "medianRank.qual")], signif(stastZ[,c("Zsummary.pres", "Zsummary.qual")], 2))) # < or equal to 10 is not preserved
        #print to file
        zsummary_table<-cbind(stasObs[,c("medianRank.pres", "medianRank.qual")], signif(stastZ[,c("Zsummary.pres", "Zsummary.qual")], 2))
        write.table(zsummary_table,paste0(save_path,"zsummary_table.tsv"),quote=F,sep="\t")

        modColors = rownames(mp2$preservation$observed[[ref]][[test]])
        moduleSizes = mp2$preservation$Z[[ref]][[test]][,1]
        plotMods = !(modColors %in% c("grey", "gold"))
        text = modColors[plotMods]
        plotData = cbind(mp2$preservation$observed[[ref]][[test]][,2], mp2$preservation$Z[[ref]][[test]][,2])
        mains = c("Preservation Median Rank", "Preservation Zsummary")
                                    
        # Plot Module Preservation Results
        sizeGrWindow(10, 5)
        pdf(paste0(save_path,'6-modulepreservation_plot.pdf'))
        par(mfrow = c(1, 2), mar = c(4.5, 4.5, 2.5, 5.1))

        for (p in 1:2) {
            min_val = min(plotData[, p], na.rm = TRUE)
            max_val = max(plotData[, p], na.rm = TRUE)

        if (p == 2) {
            if (min_val > -max_val / 10) min_val = -max_val / 10
            }

        ylim = c(min_val - 0.1 * (max_val - min_val), max_val + 0.1 * (max_val - min_val))

        plot(moduleSizes[plotMods], plotData[plotMods, p], col = 1, bg = modColors[plotMods], pch = 21, 
                main = mains[p], cex = 3.4, ylab = mains[p], xlab = "Module Size", log = "x", ylim = ylim,
                xlim = c(min(moduleSizes[plotMods]) / 2, max(moduleSizes[plotMods]) * 1.2),
                cex.lab = 1.0, cex.axis = 1.0, cex.main = 1.4)

        labelPoints(moduleSizes[plotMods], plotData[plotMods, p], text, cex = 0.5)

          # For Z summary, add threshold lines
        if (p == 2) {
            abline(h = 0)
            abline(h = 2, col = "blue", lty = 2)
            abline(h = 10, col = "darkgreen", lty = 2)
            }
        }

        dev.off()
                                    
        pdf(paste0(save_path,'7-zsummarystatistics_plot.pdf'), width = 10, height = 10)
        modColors = rownames(stastZ)
        modulesSize = mp2$quality$Z[[ref]][[test]][, 1]
        plotMods = !(modColors %in% c("grey", "gold"))

        par(mfrow = c(4, 5))
        par(mar = c(3, 3, 2, 1))
        par(mgp = c(1.6, 0.4, 0))

        num_cols = ncol(stastZ)
        for (s in 1:num_cols) {
            min_val = min(stastZ[plotMods, s], na.rm = TRUE)
            max_val = max(stastZ[plotMods, s], na.rm = TRUE)

            if (min_val > -max_val / 5) min_val = -max_val / 5

                plot(modulesSize[plotMods], stastZ[plotMods, s], col = 1, bg = modColors[plotMods], pch = 21,
                        main = colnames(stastZ)[s], cex = 1.7, ylab = colnames(stastZ)[s], xlab = "Module size",
                        log = "x", ylim = c(min_val - 0.1 * (max_val - min_val), max_val + 0.1 * (max_val - min_val)),
                        xlim = c(min(moduleSizes[plotMods]) / 2, max(moduleSizes[plotMods]) * 1.2),
                        cex.main = 1.0, cex.lab = 0.8)

                abline(h = 0)
                abline(h = 2, col = "blue", lty = 2)
                abline(h = 10, col = "darkgreen", lty = 2)
        }

        dev.off()
                                    
        # Print the Module Preservantion Stastitics Results into .tsv files
                                    
        final_genes_modules <- data.frame(gene_id = names(datExprA),
                                          module = dynamicColorsA)
        fwrite(final_genes_modules,file=paste0(save_path,"Modules_Genes.tsv"), sep="\t", row.names=FALSE,quote = F)




        ME_1A <- write_module_eigengenes(datExprA, dynamicColorsA, setA, "3a")
        kMEtable1 <- write_kME_table(datExprA, ME_1A, dynamicColorsA, setA, "4a")

        ME_2A <- write_module_eigengenes(datExprB, dynamicColorsA, setB, "3b")
        kMEtable2 <- write_kME_table(datExprB, ME_2A, dynamicColorsA, setB, "4b")
                                    
        print('############################## end')

            
        }
    
    }

Loading required package: limma



[1] "############################## start"
 Flagging genes and samples with too many missing values...
  ..step 1
 Flagging genes and samples with too many missing values...
  ..step 1
 Flagging genes and samples with too many missing values...
  ..step 1
 Flagging genes and samples with too many missing values...
  ..step 1
[1] "group_a after goodSamplesGenes(): 26881"
[2] "group_a after goodSamplesGenes(): 272"  
[1] "group_b after goodSamplesGenes(): 26881"
[2] "group_b after goodSamplesGenes(): 121"  
[1] "datExprA after goodSamplesGenes(): 272"  
[2] "datExprA after goodSamplesGenes(): 26881"
[1] "datExprB after goodSamplesGenes(): 121"  
[2] "datExprB after goodSamplesGenes(): 26881"
[1] "datExprAll : 393"   "datExprAll : 26881"
..connectivity..
..matrix multiplication (system BLAS)..
..normalization..
..done.
..connectivity..
..matrix multiplication (system BLAS)..
..normalization..
..done.
 ..cutHeight not given, setting it to 1  ===>  99% of the (truncated) height range in den

In [7]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Arch Linux

Matrix products: default
BLAS:   /usr/lib/libopenblas.so.0.3
LAPACK: /usr/lib/liblapack.so.3.11.0

locale:
 [1] LC_CTYPE=C.UTF-8    LC_NUMERIC=C        LC_TIME=C          
 [4] LC_COLLATE=C        LC_MONETARY=C       LC_MESSAGES=C      
 [7] LC_PAPER=C          LC_NAME=C           LC_ADDRESS=C       
[10] LC_TELEPHONE=C      LC_MEASUREMENT=C    LC_IDENTIFICATION=C

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] limma_3.54.2          gridExtra_2.3         latticeExtra_0.6-30  
 [4] lattice_0.20-45       flashClust_1.01-2     WGCNA_1.72-1         
 [7] fastcluster_1.2.3     dynamicTreeCut_1.63-1 data.table_1.14.8    
[10] lubridate_1.9.2       forcats_1.0.0         stringr_1.5.0        
[13] dplyr_1.1.0           purrr_1.0.1           readr_2.1.4          
[16] tidyr_1.3.0           tibble_3.1.8     